# Analysis of visual cortical neurons of mice

# What this graph is about

Data of this graph was collected from the study of the paper [[3]](README.md#references).

The aim of the graph is to understand better the connections and structor of neurons in the primary visual cortex of a mouse.

Researchers have simulated visual stimulus on some neurons and tracked where the synapsis were propagated or not.

## Data components

Three types of components were tracked:

- Characterized pyramidal neuron
- Cell body in EM volume
- Dendritic fragment

We will try to get a brief explanation of these components.

### Characterized pyramidal neuron

These are the neurons that were stimulated with two-photon calcium imaging. A specific visual 
stimulus is created with this technique.
> Every synapse starts from one of them.

### Cell body in EM volume

These are the neurons that are targeted from the synapse. With the large-scale electron microscopy (EM), like we described in the [README](README.md#scope-of-the-paper), is it possible 
to trace a portion of the local network of these neurons.

### Dendritic fragment

Part of the neuron specialized in retrieving signals from other neurons. It is different from the 
axon which is specialized in sending signals to other neurons.

> We have *fragment* because the Dendritic is highly branched, so only a fraction of it was tracked.   

## Further division of the components

We said that a **pyramidal neuron** "starts" the synapse targeting a specific neuron.

Post-Synaptic data was also collected, so the components: *Cell body in EM volume* and *Dendritic fragment* can be classified to:

- Postsynaptic excitatory target,
- Postsynaptic inhibitory target

**excitatory** and **inhibitory** means: "A target that *propagates* the signal and a target that *stops* the signal respectively."

> Since all the synapses start from the *characterized pyramidal neurons*, they do not have this further division and that attribute is marked as *NA*.
 

# Before Starting the analysis

Now that we know **what** data is collected, **why** and **how**, we can formulate questions.

This is an important step because we must know what we are looking for, otherwise we might not be able to see it if we find it.

We can split the techniques to answer the questions, and the rest of the notebook, in two categories:

1. [Descriptive analysis](#descriptive-analysis)
2. [Statistical modeling](#statistical-modeling)

> We will not use any machine learning technique, since is out of the scope of this course. 
> We will focus on the statistical aspects of Network data.


## Analysis questions

some questions to discuss later here. :)

# Descriptive analysis

In [1]:
import igraph as ig
import matplotlib.pyplot as plt

We start by importing the graph downloaded from the site [[2]](/README.md#references).

Unfortunately with this version we are using of iGraph, we can't read from an URL.

In [2]:
neurons : ig.Graph
neurons = ig.Graph.Read_GraphML("Data/mouse_visual.cortex_2.graphml")
neurons.to_undirected()

> Since the return type of `Read_GraphML` is erroneously hinted as `None` (which is not because is a `Graph`) we hint the interpreter thar `neurons` is a `Graph`, so we can use autocompletion and suggestions.

In [3]:
print(f"The Graph is bipartite: {neurons.is_bipartite()}")
print(f"Nodes can be classified as: {set(neurons.vs['type1'])}")
print(f"We can further classify as: {set(neurons.vs['type2'])}")



The Graph is bipartite: False
Nodes can be classified as: {'Cell body in EM volume', 'Characterized pyramidal neuron', 'Dendritic fragment'}
We can further classify as: {'Postsynaptic excitatory target', 'Postsynaptic inhibitory target', 'NA'}


In [4]:
neurons_list = list(neurons.vs)
for neuron in neurons_list:
    if neuron["type1"] == 'Characterized pyramidal neuron':
         if not (neuron["type2"] == 'NA'):
            print(f"Not True {neuron['id']}")
print("Done")
# Pyram -> NA
# Gli altri si "caricano e scaricano"

Done


In [5]:
for edge in neurons.es:
    print(f'{neurons.vs[edge.source]["id"]}->{neurons.vs[edge.target]["id"]}')

n1->n4
n1->n5
n2->n7
n3->n7
n1->n12
n4->n12
n1->n14
n1->n15
n1->n16
n1->n17
n1->n18
n12->n18
n12->n19
n12->n20
n12->n21
n12->n22
n12->n23
n12->n24
n12->n25
n12->n26
n12->n27
n12->n28
n12->n29
n12->n30
n12->n31
n12->n32
n12->n33
n12->n34
n9->n35
n11->n35
n12->n35
n5->n36
n10->n36
n5->n37
n5->n38
n5->n39
n5->n40
n5->n41
n5->n42
n5->n43
n5->n44
n5->n45
n5->n46
n5->n47
n5->n48
n5->n49
n5->n50
n5->n51
n5->n52
n5->n53
n5->n54
n5->n55
n5->n56
n5->n57
n5->n58
n5->n59
n6->n59
n5->n60
n6->n60
n6->n61
n6->n62
n6->n63
n6->n64
n6->n65
n6->n66
n6->n67
n6->n68
n6->n69
n6->n70
n6->n71
n6->n72
n6->n73
n6->n74
n6->n75
n6->n76
n6->n77
n6->n78
n6->n79
n6->n80
n6->n81
n2->n82
n5->n82
n9->n82
n12->n82
n9->n83
n9->n84
n9->n85
n9->n86
n9->n87
n9->n88
n9->n89
n9->n90
n9->n91
n9->n92
n9->n93
n9->n94
n9->n95
n9->n96
n9->n97
n9->n98
n9->n99
n9->n100
n9->n101
n3->n102
n5->n102
n9->n102
n9->n103
n10->n103
n9->n104
n10->n104
n9->n105
n10->n105
n10->n106
n10->n107
n10->n108
n10->n109
n10->n110
n10->n111
n10->n112
n10

In [6]:
print(neurons['info'])
print(neurons['source'])
print(neurons['sensor'])
print(neurons['DOI'])
print(neurons['region'])
print(neurons['key'])

Figure 5b
http://www.nature.com/nature/journal/v471/n7337/full/nature09802.html
Electron microscopy
doi:10.1038/nature09802
cerebral cortex
IT=inhibitory targets, CBEV=cell body in EM volume,


# Statistical Modeling